In [244]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np

### ● 不動産の一つの物件から情報を取得する。

In [245]:
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13109&sc=13110&sc=13111&sc=13112&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"
# とりあえず都心＆南部（東京都千代田区、東京都中央区、東京都港区、東京都新宿区、東京都文京区、東京都渋谷区、東京都品川区、東京都目黒区、東京都大田区、東京都世田谷区）で設定

response = requests.get(base_url)
soup     = BeautifulSoup(response.content, "lxml")
items    = soup.find(class_="cassetteitem")

In [246]:
# スクレイピングした HTML情報出力
f = open("test.txt", "w")
f.write(str(items))
f.close()

In [247]:
# 各物件情報の取得
property_name     = items.find(class_="cassetteitem_content-title").get_text() if items.find(class_="cassetteitem_content-title") else None
category          = items.find(class_="cassetteitem_content-label").span.get_text() if items.find(class_="cassetteitem_content-label") else None
address           = items.find(class_="cassetteitem_detail-col1").get_text() if items.find(class_="cassetteitem_detail-col1") else None
nearest_stations  = [station.get_text() for station in items.find_all(class_="cassetteitem_detail-text")] if items.find_all(class_="cassetteitem_detail-text") else None
construction_info = items.find(class_="cassetteitem_detail-col3").find_all("div") if items.find(class_="cassetteitem_detail-col3") else None
years_since_const = construction_info[0].get_text() if construction_info and len(construction_info) > 0 else None
number_of_floors  = construction_info[1].get_text() if construction_info and len(construction_info) > 1 else None
floor_number_td   = items.select_one("tr.js-cassette_link .cassetteitem_other-col03")
floor_number      = floor_number_td.get_text(strip=True) if floor_number_td else None
rent_info         = items.select_one(".cassetteitem_other tbody .js-cassette_link")
rent_admin_fee    = " / ".join([item.get_text(strip=True) for item in rent_info.select(".cassetteitem_price--rent, .cassetteitem_price--administration")]) if rent_info else None
deposit_gratuity  = " / ".join([price.get_text(strip=True) for price in rent_info.select(".cassetteitem_price--deposit, .cassetteitem_price--gratuity")]) if rent_info else None
layout_total_area = " / ".join([detail.get_text(strip=True) for detail in rent_info.select(".cassetteitem_madori, .cassetteitem_menseki")]) if rent_info else None


# 各物件情報の表示
print("物件名称 (Property Name):", property_name)
print("カテゴリー (Category):", category)
print("住所 (Address):", address)
print("最寄り駅 (Nearest Stations):", nearest_stations)
print("築年数 (Years Since Construction):", years_since_const)
print("階建 (Number of Floors):", number_of_floors)
print("階数 (Floor Number):", floor_number)
print("賃料/管理費 (Rent/Administration Fee):", rent_admin_fee)
print("敷金/礼金 (Deposit/Key Money):", deposit_gratuity)
print("間取り/占有面積 (Layout/Total Area):", layout_total_area)


物件名称 (Property Name): グリーンヒルズ泰明
カテゴリー (Category): 賃貸マンション
住所 (Address): 東京都文京区本郷６
最寄り駅 (Nearest Stations): ['都営三田線/春日駅 歩6分', '東京メトロ南北線/東大前駅 歩7分', '東京メトロ南北線/後楽園駅 歩10分']
築年数 (Years Since Construction): 築22年
階建 (Number of Floors): 3階建
階数 (Floor Number): None
賃料/管理費 (Rent/Administration Fee): 9.1万円 / 3000円
敷金/礼金 (Deposit/Key Money): 9.1万円 / 9.1万円
間取り/占有面積 (Layout/Total Area): 1K / 26.55m2


In [248]:
# 物件画像・間取り画像・詳細URLの取得
property_image_element = items.find(class_="cassetteitem_object-item")
property_image_url = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

floor_plan_image_element = items.find(class_="casssetteitem_other-thumbnail")
floor_plan_image_url = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

property_link_element = items.select_one("a[href*='/chintai/jnc_']")
property_link = "https://suumo.jp" + property_link_element['href'] if property_link_element else None ## 不動産サイトから詳細URLリンクを読み解き作成

# 物件画像・間取り画像・詳細URLの表示
print("物件画像 URL (Property Image URL):", property_image_url)
print("間取り情報画像 URL (Floor Plan Image URL):", floor_plan_image_url)
print("物件リンク (Property Link):", property_link)

物件画像 URL (Property Image URL): https://img01.suumo.com/front/gazo/fr/bukken/427/100363338427/100363338427_gw.jpg
間取り情報画像 URL (Floor Plan Image URL): https://img01.suumo.com/front/gazo/fr/bukken/290/100382398290/100382398290_co.jpg
物件リンク (Property Link): https://suumo.jp/chintai/jnc_000090702474/?bc=100382398290


### ● 不動産の複数ページから情報を取得する。

In [249]:
# 基本URLと最大ページ数の設定
base_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13109&sc=13110&sc=13111&sc=13112&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"
# とりあえず都心＆南部（東京都千代田区、東京都中央区、東京都港区、東京都新宿区、東京都文京区、東京都渋谷区、東京都品川区、東京都目黒区、東京都大田区、東京都世田谷区）で設定

max_page = 10  # 最大ページ数

all_data = []

for page in range(1, max_page + 1):
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    items = soup.findAll("div", {"class": "cassetteitem"})


    print("page", page, "items", len(items))

    for item in items:
        base_data = {}
        base_data["名称"]     = item.find("div", {"class": "cassetteitem_content-title"}).get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-title"}) else None
        base_data["カテゴリ"] = item.find("div", {"class": "cassetteitem_content-label"}).span.get_text(strip=True) if item.find("div", {"class": "cassetteitem_content-label"}) else None
        base_data["アドレス"] = item.find("li", {"class": "cassetteitem_detail-col1"}).get_text(strip=True) if item.find("li", {"class": "cassetteitem_detail-col1"}) else None
        
        # 駅のアクセス情報をまとめて取得
        base_data["アクセス"] = ", ".join([station.get_text(strip=True) for station in item.findAll("div", {"class": "cassetteitem_detail-text"})])

        construction_info = item.find("li", {"class": "cassetteitem_detail-col3"}).find_all("div") if item.find("li", {"class": "cassetteitem_detail-col3"}) else None
        base_data["築年数"] = construction_info[0].get_text(strip=True) if construction_info and len(construction_info) > 0 else None
        base_data["構造"] = construction_info[1].get_text(strip=True) if construction_info and len(construction_info) > 1 else None

        tbodys = item.find("table", {"class": "cassetteitem_other"}).findAll("tbody")

        for tbody in tbodys:
            data = base_data.copy()
            # 階数情報の正確な取得
            floor_info = tbody.find_all("td")[2].get_text(strip=True) if len(tbody.find_all("td")) > 2 else None
            data["階数"]   = floor_info
            data["家賃"]   = tbody.select_one(".cassetteitem_price--rent").get_text(strip=True) if tbody.select_one(".cassetteitem_price--rent") else None
            data["管理費"] = tbody.select_one(".cassetteitem_price--administration").get_text(strip=True) if tbody.select_one(".cassetteitem_price--administration") else None
            data["敷金"]   = tbody.select_one(".cassetteitem_price--deposit").get_text(strip=True) if tbody.select_one(".cassetteitem_price--deposit") else None
            data["礼金"]   = tbody.select_one(".cassetteitem_price--gratuity").get_text(strip=True) if tbody.select_one(".cassetteitem_price--gratuity") else None
            data["間取り"] = tbody.select_one(".cassetteitem_madori").get_text(strip=True) if tbody.select_one(".cassetteitem_madori") else None
            data["面積"]   = tbody.select_one(".cassetteitem_menseki").get_text(strip=True) if tbody.select_one(".cassetteitem_menseki") else None

            # 物件画像・間取り画像・詳細URLの取得を最後に行う
            property_image_element = item.find(class_="cassetteitem_object-item")
            data["物件画像URL"] = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

            floor_plan_image_element = item.find(class_="casssetteitem_other-thumbnail")
            data["間取画像URL"] = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

            property_link_element = item.select_one("a[href*='/chintai/jnc_']")
            data["物件詳細URL"] = "https://suumo.jp" +property_link_element['href'] if property_link_element else None ## 不動産サイトから詳細URLリンクを読み解き作成

            all_data.append(data)    


page 1 items 30
page 2 items 30
page 3 items 30
page 4 items 30
page 5 items 30
page 6 items 30
page 7 items 30
page 8 items 30
page 9 items 30
page 10 items 30


In [250]:
df = pd.DataFrame(all_data)
df = df.drop_duplicates() # 重複データの削除
df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
0,グリーンヒルズ泰明,賃貸マンション,東京都文京区本郷６,"都営三田線/春日駅 歩6分, 東京メトロ南北線/東大前駅 歩7分, 東京メトロ南北線/後楽園...",築22年,3階建,1階,9.1万円,3000円,9.1万円,9.1万円,1K,26.55m2,https://img01.suumo.com/front/gazo/fr/bukken/4...,https://img01.suumo.com/front/gazo/fr/bukken/2...,https://suumo.jp/chintai/jnc_000090702474/?bc=...
1,グリーンヒルズ泰明,賃貸マンション,東京都文京区本郷６,"都営三田線/春日駅 歩6分, 東京メトロ南北線/東大前駅 歩7分, 東京メトロ南北線/後楽園...",築22年,3階建,3階,9.3万円,3000円,9.3万円,9.3万円,1K,24.31m2,https://img01.suumo.com/front/gazo/fr/bukken/4...,https://img01.suumo.com/front/gazo/fr/bukken/2...,https://suumo.jp/chintai/jnc_000090702474/?bc=...


In [251]:
df.tail(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL
252,パークアクシス御茶ノ水ヒルトップ,賃貸マンション,東京都千代田区神田駿河台２,"ＪＲ中央線/御茶ノ水駅 歩5分, 都営三田線/水道橋駅 歩7分, 東京メトロ千代田線/新御茶...",築6年,14階建,4階,20.6万円,15000円,20.6万円,30.9万円,1LDK,44.64m2,https://img01.suumo.com/front/gazo/fr/bukken/6...,https://img01.suumo.com/front/gazo/fr/bukken/8...,https://suumo.jp/chintai/jnc_000089913449/?bc=...
253,パークアクシス御茶ノ水ヒルトップ,賃貸マンション,東京都千代田区神田駿河台２,"ＪＲ中央線/御茶ノ水駅 歩5分, 都営三田線/水道橋駅 歩7分, 東京メトロ千代田線/新御茶...",築6年,14階建,13階,26.8万円,15000円,26.8万円,53.6万円,2LDK,56.56m2,https://img01.suumo.com/front/gazo/fr/bukken/6...,https://img01.suumo.com/front/gazo/fr/bukken/8...,https://suumo.jp/chintai/jnc_000089913449/?bc=...


### ● Googleスプレッドシートへの書き込み・読み込み。

In [252]:
# google スプレッドシート 書き込み・読み込み
import gspread
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe

In [253]:
from dotenv import load_dotenv
import os

# 環境変数の読み込み
load_dotenv()

# 環境変数から認証情報を取得
SPREADSHEET_ID = "1_JALV0dH6ROPgKKMgjD_yizCqGzY_woSLxXxAVjaftw"
PRIVATE_KEY_PATH = "/Users/simon/Realestate_Search/齋藤作業用/realestatesearch-yokopisans160-47c75af30d9f.json"

# 一旦齋藤の情報をベタで入れています。

In [254]:
PRIVATE_KEY_PATH

'/Users/simon/Realestate_Search/齋藤作業用/realestatesearch-yokopisans160-47c75af30d9f.json'

In [255]:
# googleスプレッドシートの認証 jsonファイル読み込み(key値はGCPから取得)
SP_CREDENTIAL_FILE = PRIVATE_KEY_PATH

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    SP_CREDENTIAL_FILE,
    scopes=scopes
)
gc = gspread.authorize(credentials)


SP_SHEET_KEY = SPREADSHEET_ID # d/〇〇/edit の〇〇部分
sh  = gc.open_by_key(SP_SHEET_KEY)

In [256]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'tech0' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, df)

In [257]:
# 不動産データの取得
SP_SHEET     = 'tech0' # sheet名
worksheet = sh.worksheet(SP_SHEET) # シートのデータ取得
pre_data  = worksheet.get_all_values()
col_name = pre_data[0][:]
new_df = pd.DataFrame(pre_data[1:], columns=col_name) # 一段目をカラム、以下データフレームで取得

In [258]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   名称       233 non-null    object
 1   カテゴリ     233 non-null    object
 2   アドレス     233 non-null    object
 3   アクセス     233 non-null    object
 4   築年数      233 non-null    object
 5   構造       233 non-null    object
 6   階数       233 non-null    object
 7   家賃       233 non-null    object
 8   管理費      233 non-null    object
 9   敷金       233 non-null    object
 10  礼金       233 non-null    object
 11  間取り      233 non-null    object
 12  面積       233 non-null    object
 13  物件画像URL  233 non-null    object
 14  間取画像URL  233 non-null    object
 15  物件詳細URL  233 non-null    object
 16  区        233 non-null    object
 17  市町       233 non-null    object
dtypes: object(18)
memory usage: 32.9+ KB


In [259]:
new_df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL,区,市町
0,グリーンヒルズ泰明,賃貸マンション,東京都文京区本郷６,"都営三田線/春日駅 歩6分, 東京メトロ南北線/東大前駅 歩7分, 東京メトロ南北線/後楽園...",築22年,3階建,1階,9.1万円,3000円,9.1万円,9.1万円,1K,26.55m2,https://img01.suumo.com/front/gazo/fr/bukken/4...,https://img01.suumo.com/front/gazo/fr/bukken/2...,https://suumo.jp/chintai/jnc_000090702474/?bc=...,文京区,本郷
1,グリーンヒルズ泰明,賃貸マンション,東京都文京区本郷６,"都営三田線/春日駅 歩6分, 東京メトロ南北線/東大前駅 歩7分, 東京メトロ南北線/後楽園...",築22年,3階建,3階,9.3万円,3000円,9.3万円,9.3万円,1K,24.31m2,https://img01.suumo.com/front/gazo/fr/bukken/4...,https://img01.suumo.com/front/gazo/fr/bukken/2...,https://suumo.jp/chintai/jnc_000090702474/?bc=...,文京区,本郷


In [260]:
new_df['築年数'] = new_df["築年数"].apply( lambda x: 0 if x=='新築' else int(re.split('[築年]', x )[1]) )

In [261]:
def get_most_floor(x):
    if ('階建' not in x) :
        return np.nan
    elif('B' not in x) :
        list = re.findall(r'(\d+)階建',str(x))
        list = map(int, list)
        min_value = min(list)
        return min_value

new_df['構造'] = new_df['構造'].apply(get_most_floor)
print(new_df['構造'].head(5))

0    3
1    3
2    3
3    3
4    2
Name: 構造, dtype: int64


In [262]:
def get_floor(x):
    if ('階' not in x) :
        return np.nan
    elif('B' not in x) :
        list = re.findall(r'(\d+)階',str(x))
        # time_listを数値型に変換
        list = map(int, list)
        # time_listの最小値をmin_valueに代入
        min_value = min(list)
        return min_value
    else:
        list = re.findall(r'(\d+)階',str(x))
        # time_listを数値型に変換
        list = map(int, list)
        # time_listの最小値をmin_valueに代入
        min_value = -1*min(list)
        return min_value

new_df['階数'] = new_df['階数'].apply(get_floor)
print(new_df['階数'].head(5))

0    1.0
1    3.0
2    3.0
3    3.0
4    NaN
Name: 階数, dtype: float64


In [263]:
def change_fee(x):
    if ('万円' not in x) :
        return np.nan
    else:
        return float(x.split('万円')[0])

new_df['家賃'] = new_df['家賃'].apply(change_fee)
new_df['敷金'] = new_df['敷金'].apply(change_fee)
new_df['礼金'] = new_df['礼金'].apply(change_fee)

In [264]:
new_df.head(5)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,礼金,間取り,面積,物件画像URL,間取画像URL,物件詳細URL,区,市町
0,グリーンヒルズ泰明,賃貸マンション,東京都文京区本郷６,"都営三田線/春日駅 歩6分, 東京メトロ南北線/東大前駅 歩7分, 東京メトロ南北線/後楽園...",22,3,1.0,9.1,3000円,9.1,9.1,1K,26.55m2,https://img01.suumo.com/front/gazo/fr/bukken/4...,https://img01.suumo.com/front/gazo/fr/bukken/2...,https://suumo.jp/chintai/jnc_000090702474/?bc=...,文京区,本郷
1,グリーンヒルズ泰明,賃貸マンション,東京都文京区本郷６,"都営三田線/春日駅 歩6分, 東京メトロ南北線/東大前駅 歩7分, 東京メトロ南北線/後楽園...",22,3,3.0,9.3,3000円,9.3,9.3,1K,24.31m2,https://img01.suumo.com/front/gazo/fr/bukken/4...,https://img01.suumo.com/front/gazo/fr/bukken/2...,https://suumo.jp/chintai/jnc_000090702474/?bc=...,文京区,本郷
2,グリーンヒルズ泰明,賃貸マンション,東京都文京区本郷６,"都営三田線/春日駅 歩6分, 東京メトロ南北線/東大前駅 歩7分, 東京メトロ南北線/後楽園...",22,3,3.0,100.0,-,200.0,100.0,6LDK,198.2m2,https://img01.suumo.com/front/gazo/fr/bukken/4...,https://img01.suumo.com/front/gazo/fr/bukken/2...,https://suumo.jp/chintai/jnc_000090702474/?bc=...,文京区,本郷
3,グリーンヒルズ泰明,賃貸マンション,東京都文京区本郷６,"都営三田線/春日駅 歩6分, 東京メトロ南北線/東大前駅 歩7分, 東京メトロ南北線/後楽園...",22,3,3.0,100.0,-,200.0,100.0,6LDK,198.2m2,https://img01.suumo.com/front/gazo/fr/bukken/4...,https://img01.suumo.com/front/gazo/fr/bukken/2...,https://suumo.jp/chintai/jnc_000090702474/?bc=...,文京区,本郷
4,ベルクレスト用賀,賃貸一戸建て,東京都世田谷区用賀２,"東急田園都市線/用賀駅 歩7分, 東急田園都市線/桜新町駅 歩10分, 東急大井町線/上野毛...",25,2,NaN,75.0,-,150.0,75.0,5SLDK,225m2,https://img01.suumo.com/front/gazo/fr/bukken/3...,https://img01.suumo.com/front/gazo/fr/bukken/3...,https://suumo.jp/chintai/jnc_000090835861/?bc=...,世田谷区,用賀


In [265]:
def change_fee2(x):
    if ('円' not in x) :
        return np.nan
    else:
        return float(x.split('円')[0])


new_df['管理費'] = new_df['管理費'].apply(change_fee2)

In [266]:
new_df['面積'] = new_df['面積'].apply(lambda x: float(x[:-2]))

In [267]:
new_df['区'] = new_df["アドレス"].apply(lambda x : x[x.find("都")+1:x.find("区")+1])

In [268]:
new_df['市町'] = new_df["アドレス"].apply(lambda x : x[x.find("区")+1 :-1])

In [269]:
def split_access(row):
    accesses = row['アクセス'].split(', ')
    results = {}

    for i, access in enumerate(accesses, start=1):
        if i > 3:
            break  # 最大3つのアクセス情報のみを考慮

        parts = access.split('/')
        if len(parts) == 2:
            line_station, walk = parts
            # ' 歩'で分割できるか確認
            if ' 歩' in walk:
                station, walk_min = walk.split(' 歩')
                # 歩数の分の数値だけを抽出
                walk_min = int(re.search(r'\d+', walk_min).group())
            else:
                station = None
                walk_min = None
        else:
            line_station = access
            station = walk_min = None

        results[f'アクセス①{i}線路名'] = line_station
        results[f'アクセス①{i}駅名'] = station
        results[f'アクセス①{i}徒歩(分)'] = walk_min

    return pd.Series(results)

# 新しい列をデータフレームに適用
new_df = new_df.join(new_df.apply(split_access, axis=1))

In [270]:
new_df.head(2)

,名称,カテゴリ,アドレス,アクセス,築年数,構造,階数,家賃,管理費,敷金,...,市町,アクセス①1線路名,アクセス①1駅名,アクセス①1徒歩(分),アクセス①2線路名,アクセス①2駅名,アクセス①2徒歩(分),アクセス①3線路名,アクセス①3駅名,アクセス①3徒歩(分)
0,グリーンヒルズ泰明,賃貸マンション,東京都文京区本郷６,"都営三田線/春日駅 歩6分, 東京メトロ南北線/東大前駅 歩7分, 東京メトロ南北線/後楽園...",22,3,1.0,9.1,3000.0,9.1,...,本郷,都営三田線,春日駅,6,東京メトロ南北線,東大前駅,7,東京メトロ南北線,後楽園駅,10.0
1,グリーンヒルズ泰明,賃貸マンション,東京都文京区本郷６,"都営三田線/春日駅 歩6分, 東京メトロ南北線/東大前駅 歩7分, 東京メトロ南北線/後楽園...",22,3,3.0,9.3,3000.0,9.3,...,本郷,都営三田線,春日駅,6,東京メトロ南北線,東大前駅,7,東京メトロ南北線,後楽園駅,10.0


In [271]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 27 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   名称           233 non-null    object 
 1   カテゴリ         233 non-null    object 
 2   アドレス         233 non-null    object 
 3   アクセス         233 non-null    object 
 4   築年数          233 non-null    int64  
 5   構造           233 non-null    int64  
 6   階数           232 non-null    float64
 7   家賃           233 non-null    float64
 8   管理費          212 non-null    float64
 9   敷金           215 non-null    float64
 10  礼金           193 non-null    float64
 11  間取り          233 non-null    object 
 12  面積           233 non-null    float64
 13  物件画像URL      233 non-null    object 
 14  間取画像URL      233 non-null    object 
 15  物件詳細URL      233 non-null    object 
 16  区            233 non-null    object 
 17  市町           233 non-null    object 
 18  アクセス①1線路名    233 non-null    object 
 19  アクセス①1駅名

### ● 緯度経度情報取得

In [280]:
# 不動産データの取得
SP_SHEET     = 'tech0' # sheet名
worksheet = sh.worksheet(SP_SHEET) # シートのデータ取得
pre_data  = worksheet.get_all_values()
col_name = pre_data[0][:]
new_df = pd.DataFrame(pre_data[1:], columns=col_name) # 一段目をカラム、以下データフレームで取得

In [281]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import pandas as pd
import time

# ジオコーダーの初期化
geolocator = Nominatim(user_agent="your_app_name", timeout=10)

current_count = 0
total_count = len(new_df['アドレス'])

# 住所から緯度と経度を取得する関数
def get_lat_lon(address, retries=3):
    global current_count
    current_count += 1
    
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address)
            if location:
                print(f"{current_count}/{total_count} 件目実施中 結果: {location.latitude}, {location.longitude}")
                return location.latitude, location.longitude
            else:
                print(f"{current_count}/{total_count} 件目実施中 結果: 住所が見つかりません")
                return None, None
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Error retrieving location for address {address}: {e}. Retrying ({attempt + 1}/{retries})...")
            time.sleep(1)  # リトライの前に1秒待つ

    print(f"Failed to retrieve location for address {address} after {retries} retries")
    return None, None

# 新しい列を作成してデータフレームに追加
new_df['緯度'], new_df['経度'] = zip(*new_df['アドレス'].apply(get_lat_lon))

# 結果の表示
print(new_df.head())

# 単一の住所のテスト
address = "東京都江戸川区篠崎町７"
location = geolocator.geocode(address)
if location:
    print(f"テストアドレスの結果: {location.latitude}, {location.longitude}")
else:
    print("テストアドレスの結果: 住所が見つかりません")

1/233 件目実施中 結果: 35.7127475, 139.7593076
2/233 件目実施中 結果: 35.7127475, 139.7593076
3/233 件目実施中 結果: 35.7127475, 139.7593076
4/233 件目実施中 結果: 35.7127475, 139.7593076
5/233 件目実施中 結果: 35.62602699999999, 139.63657948130992
6/233 件目実施中 結果: 35.6565057, 139.7724876309287
7/233 件目実施中 結果: 35.6565057, 139.7724876309287
8/233 件目実施中 結果: 35.6565057, 139.7724876309287
9/233 件目実施中 結果: 35.6565057, 139.7724876309287
10/233 件目実施中 結果: 35.6565057, 139.7724876309287
11/233 件目実施中 結果: 35.6565057, 139.7724876309287
12/233 件目実施中 結果: 35.6565057, 139.7724876309287
13/233 件目実施中 結果: 35.6565057, 139.7724876309287
14/233 件目実施中 結果: 35.6565057, 139.7724876309287
15/233 件目実施中 結果: 35.6565057, 139.7724876309287
16/233 件目実施中 結果: 35.6565057, 139.7724876309287
17/233 件目実施中 結果: 35.6565057, 139.7724876309287
18/233 件目実施中 結果: 35.6565057, 139.7724876309287
19/233 件目実施中 結果: 35.6565057, 139.7724876309287
20/233 件目実施中 結果: 35.6565057, 139.7724876309287
21/233 件目実施中 結果: 35.6565057, 139.7724876309287
22/233 件目実施中 結果: 35.6565057, 139.77248

In [282]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'tech0_2' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, new_df)